# Computational control - Tutorial 7: Identification

In this tutorial, we illustrate how to identify a linear time-invariant system via its impulse response. We use the Kalman-Ho algorithm, a foundational technique in system identification (first presented [here](https://www.degruyter.com/document/doi/10.1524/auto.1966.14.112.545/html)).


### Package imports

In [28]:
import numpy  as np                        # For linear algebra
np.set_printoptions(precision=1)           # Set nice printing format

### Problem setup

Consider a linear time-invariant system described by the equations

$$
\begin{align}
x_{t+1} &= A x_t + B u_t ,\\
y_{t}   &= C x_t + D u_t,
\end{align}
$$

with $x_t \in \mathbb{R}^n$, $u_t \in \mathbb{R}^m$, $y_t \in \mathbb{R}^p$  and constant matrices $A \in \mathbb{R}^{n\times n}$, $B \in \mathbb{R}^{n\times m}$, $C \in \mathbb{R}^{p\times n}$, and $D \in \mathbb{R}^{p\times m}$.


For illustration, we focus on the specific linear system defined by the matrices

$$
\begin{align}
A =
\begin{bmatrix}
1 & 0 \\
0 & 2
\end{bmatrix}, \quad
B =  I , \quad
C =  I, \quad
D = 0 .
\end{align}
$$


In [29]:
# System parameters
A = np.array([[1.0, 0.0],                  # A - State-space matrix
              [0.0, 2.0]])                 #
B = np.eye(2)                              # B - State-space matrix
C = np.eye(2)                              # C - State-space matrix
D = np.zeros([2,2])                        # D - State-space matrix

# Integer invariants
n = A.shape[0]                             # n = number of states
m = B.shape[1]                             # m = number of inputs
p = C.shape[0]                             # p = number of output

### Data collection

The Kalman-Ho algorithm requires the sequence of Markov parameters to determine the system matrices $A,B,C,D$ from the impulse response of the system.

The _Markov parameters_ are defined as

$$
 G_t =
\left \{
\begin{array}[rcl]
\, D, &\text{ for } t=0,\\
CA^{t-1}B , &\text{ for } t=1, 2, \ldots, T .
\end{array}
\right.
$$

The sequence can be obtained directly from the response of the system to a (matrix) _unit impulse_ of length $T+1$, defined as

$$
u_t =
\left \{
\begin{array}[rcl]
\, I &\text{ for } t=0\\
\mathbb{0} &\text{ for } t=1, 2, \ldots, T
\end{array}
\right.
$$

starting from zero initial conditions. In the following script, we select $T=8.$


Next, you are asked to perform the following:

1. **Record the impulse response**: Apply a (matrix) unit impulse sequence $u_t$ of length $T+1$ defined as above to the system and collect the output $y_t$ into a list called 'impulse_response'.

2. **Determine the feedthrough term**: Set $D_{\text{estimate}} = y_0$ to determine the feedthrough term.

3. **Define a list of Markov parameters**: Collect the output responses
 $ y_1, y_2, y_3, \ldots, y_{T-1} $ (excluding the first and last elements) into a list of Markov parameters called 'markov_parameters'.

4. **Construct a list of shifted Markov parameters**: Collect the output responses  $y_2, y_3, y_4, \ldots, y_{T}$ (by shifting by one time step forward) and define a list called 'markov_parameters_shifted'.

You may find useful the following methods:

 - `.append(x) ` (to add x to a list)
 - `.pop(0)` (to obtain and simultaneously remove the first element of a list)




In [30]:
# Data collection parameters
T = 8                                            # Length of unit impulse
impulse = [np.eye(m)]+[np.zeros((m,m))]*T        # Unit impulse (of length T+1)
x = np.zeros((n,n))                              # Initial condition (zero)

# Data collection
markov_parameters = []                           # Impulse response
impulse_response = []                            # Markov parameters
markov_parameters_shifted = []                   # Shifted Markov parameters


##################################################
#       Please implement the following           #
##################################################
# 1. Collect data from system using "impulse"    #
# 2. Estimate the feedthrough term (D matrix)    #
# 3. Define a list of Markov parameters          #
# 4. Define a list of shifted Markov parameters  #
##################################################

for t in range(T+1):
    u = impulse[t]
    y = C @ x + D @ u
    x = A @ x + B @ u
    impulse_response.append(y)

D_estimate = impulse_response[0]
impulse_response.pop(0)

markov_parameters = impulse_response[:-1]
markov_parameters_shifted = impulse_response[1:]



# Print results
print("\n Markov parameters:\n")                 # Print Markov parameters ...
print(*markov_parameters, sep='\n')              #
print("\n Markov parameters (shifted):\n")       # ... and their shifts
print(*markov_parameters_shifted, sep='\n')      #


 Markov parameters:

[[1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 2.]]
[[1. 0.]
 [0. 4.]]
[[1. 0.]
 [0. 8.]]
[[ 1.  0.]
 [ 0. 16.]]
[[ 1.  0.]
 [ 0. 32.]]
[[ 1.  0.]
 [ 0. 64.]]

 Markov parameters (shifted):

[[1. 0.]
 [0. 2.]]
[[1. 0.]
 [0. 4.]]
[[1. 0.]
 [0. 8.]]
[[ 1.  0.]
 [ 0. 16.]]
[[ 1.  0.]
 [ 0. 32.]]
[[ 1.  0.]
 [ 0. 64.]]
[[  1.   0.]
 [  0. 128.]]


### Hankel matrices

The Kalman-Ho algorithm uses the sequence of Markov parameters to construct a Hankel matrix and a shifted Hankel matrix. These matrices, in turn, are then used to determine the system matrices.

The _Hankel matrix_ is defined as

$$
{H}_{k,\ell}
=
\begin{bmatrix}
G_{1}   & G_{2} & \cdots & G_{\ell} \\
G_{2}   & G_{3} & \cdots & G_{\ell+1} \\
\vdots  & \vdots  & \ddots & \vdots    \\
G_{k}   & G_{k+1} & \cdots & G_{k+\ell-1}
\end{bmatrix}
$$

and the _shifted Hankel matrix_ is defined as

$$
{H}_{k,\ell}^{\uparrow}
=
\begin{bmatrix}
G_{2}   & G_{3} & \cdots & G_{\ell+1} \\
G_{3}   & G_{4} & \cdots & G_{\ell+2} \\
\vdots  & \vdots  & \ddots & \vdots    \\
G_{k+1}   & G_{k+2} & \cdots & G_{k+\ell}
\end{bmatrix}
$$

where $k$ and $\ell$ are positive integers such that $k+\ell\le T.$ In this script, we select $k=3$ and $\ell=5$.

Next, you are asked to perform the following:

1. **Construct Hankel matrix**: Arrange the Markov parameters collected in the list 'markov_parameters' into a Hankel matrix named 'Hankel'.

2. **Construct the shifted Hankel matrix**: Similarly, arrange the Markov parameters collected in the list 'markov_parameters_shifted' into a Hankel matrix named 'Hankel_shifted'.

In [35]:
# Hankel matrices - parameters
k = 3                                            # Number of block rows
l = 5                                            # Number of block columns

if (k+l<T+1):
  Hankel = np.empty((p*k,m*l))                   # Hankel matrix
  Hankel_shifted = np.empty((p*k,m*l))           # Shifted Hankel matrix
else:
    raise Exception('Need more data')            # Need more data

##################################################
#       Please implement the following           #
##################################################
# 1. Construct the Hankel matrix                 #
# 2. Construct the shifted Hankel matrix         #
##################################################

for i in range(k):
  for j in range(l):
    Hankel[i*p:(i+1)*p,j*m:(j+1)*m] = \
            markov_parameters[i+j]               # Define Hankel matrix

for i in range(k):
  for j in range(l):
    Hankel_shifted[i*p:(i+1)*p,j*m:(j+1)*m] = \
              markov_parameters_shifted[i+j]     # Define shifted Hankel matrix
 



    


# Print results
print("Hankel = \n",Hankel)                      # Print Hankel matrix
print("Hankel_shifted = \n",Hankel_shifted)      # Print shifted Hankel matrix

Hankel = 
 [[ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
 [ 0.  1.  0.  2.  0.  4.  0.  8.  0. 16.]
 [ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
 [ 0.  2.  0.  4.  0.  8.  0. 16.  0. 32.]
 [ 1.  0.  1.  0.  1.  0.  1.  0.  1.  0.]
 [ 0.  4.  0.  8.  0. 16.  0. 32.  0. 64.]]
Hankel_shifted = 
 [[  1.   0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  0.   2.   0.   4.   0.   8.   0.  16.   0.  32.]
 [  1.   0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  0.   4.   0.   8.   0.  16.   0.  32.   0.  64.]
 [  1.   0.   1.   0.   1.   0.   1.   0.   1.   0.]
 [  0.   8.   0.  16.   0.  32.   0.  64.   0. 128.]]


### State space dimension

The Kalman-Ho algorithm requires an estimate of the dimension of the system ($n$). The state space dimension can be retrieved directly by computing the rank of the Hankel matrix.

In [36]:
# Estimate state space dimension
N = np.linalg.matrix_rank(Hankel)
print("rank(Hankel) = ",N,"\n")

rank(Hankel) =  2 



### Kalman-Ho algorithm

We now have all the main ingredients to implement the Kalman-Ho algorithm.

Recall that the algorithm involves the following steps:

1. Computing a **thin** Singular Value Decomposition (SVD) of the Hankel matrix
 $${H}_{k,\ell} = U_1 \Sigma_1 V_1^{\mathsf{T}}.$$
2. Computing the observability matrix as
 $$\mathbf{O}_k = U_1 \Sigma_1^{1/2} $$
 and the controllability matrix as
 $$\mathbf{C}_\ell = \Sigma_1^{1/2} V_1^{\mathsf{T}}.$$
3. Computing the system matrices as
$$
\begin{align}
A &= \mathbf{O}_k^{\dagger}{H}_{k,\ell}^{\uparrow}\mathbf{C}_\ell^{\dagger} \\
B &= \text{first } m \text{ columns of } \mathbf{C}_\ell \\
C &= \text{first } p \text{ rows of } \mathbf{O}_k \\
D &= G_0 \\
\end{align}
$$
where $M^{\dagger}$ is the Moore-Penrose inverse of the matrix $M$.


Next, you are asked to implement the Kalman-Ho algorithm. You may find useful the following functions/methods:

 - `np.linalg.svd` (to compute SVD roots)
 - `np.sqrt` (to compute square roots)
 - `np.linalg.pinv` (to compute the Moore-Penrose inverse)
 - `.round(decimals=12)` (to round "small" numbers)






In [41]:
## Kalman-Ho algorithm
from scipy.linalg import sqrtm

##################################################
#       Please implement the following           #
##################################################
# 1. Compute thin SVD of the Hankel matrix       #
# 2. Define observability/controllability matrix #
# 3. Compute A,B,C,D matrices                    #
##################################################

U, S, V = np.linalg.svd(Hankel, full_matrices=False)
S = S[0:N]
Lambda_half = np.diag(np.sqrt(S))
U = U[:,0:N]
V = V[0:n,:]

O = U @ Lambda_half

C = Lambda_half @ V

C_estimate = O[:2, :2]

B_estimate = C[:2, :2]

A_estimate = (np.linalg.pinv(O) @ Hankel_shifted @ np.linalg.pinv(C)).round(decimals=12)


# Print results
print("\n A_estimate:\n")                 # Print system matrices
print(*A_estimate, sep='\n')              #
print("\n B_estimate:\n")                 #
print(*B_estimate, sep='\n')              #
print("\n C_estimate:\n")                 #
print(*C_estimate, sep='\n')              #
print("\n D_estimate:\n")                 #
print(*D_estimate, sep='\n')              #


 A_estimate:

[2. 0.]
[0. 1.]

 B_estimate:

[2.9e-17 5.0e-01]
[0.9 0. ]

 C_estimate:

[0.  1.1]
[2. 0.]

 D_estimate:

[0. 0.]
[0. 0.]


### Question
Are the system matrices identical to those used for data generation? Does it matter?

They are not the same, but transformed by a simmilarity transformation. 
However, it does not matter, as it has the samit input/output behaviour